In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
#imports
import torch
from torch.utils.data import Dataset
import numpy as np
from src.updated_model import HarvestModel
from torch.utils.data import DataLoader
import src.preprocessing as pre
from torch.utils.data import random_split
import torch.nn as nn
import torch.optim as optim

In [3]:
from src.updated_dataset import create_dataset
from src.updated_utils import create_model

In [4]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset

In [5]:
meta, y, mapping_dict = pre.load_data('../data/meta.json','../data/y.csv','../data/mappings.json')

In [6]:
dataset = create_dataset(meta, y, mapping_dict)

In [ ]:
results = np.zeros((len(dataset),16))
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)): #type: ignore
    train_dataset = Subset(dataset, train_idx) #type: ignore
    val_dataset = Subset(dataset, val_idx) #type: ignore

    model, val_loader = create_model(train_dataset, val_dataset, 1)

    results = []
    model.eval()
    criterion = nn.MSELoss()
    with torch.no_grad():
        for batch in val_loader:
            features, ranch_id, class_id, type_id, variety_id, climate_data, y, bounds, idx = batch
            kilo_ranges = torch.arange(4,20)
            batch_size = len(y)
            log_kilos = torch.log1p(y) 
            week_numbers = torch.arange(0, 20).unsqueeze(0).repeat(batch_size,1)
            inputs = torch.stack([y, log_kilos, week_numbers], dim=2)

            for i,kilo_range in enumerate(kilo_ranges):
                
                kilo_inputs = inputs[:,:kilo_range.item(),:]
                outputs, clamp = model(features, ranch_id, class_id, type_id, variety_id, climate_data, kilo_inputs)







Epoch [1/200], Loss: 8715783.6732


RuntimeError: a Tensor with 32 elements cannot be converted to Scalar

In [ ]:
results = []
model.eval()
criterion = nn.MSELoss()
with torch.no_grad():
    for batch in val_loader:
        features, ranch_id, class_id, type_id, variety_id, climate_data, y, bounds = batch
        kilo_ranges = torch.arange(4,20,4)
        row_results = []
        batch_size = len(y)
        log_kilos = torch.log1p(y) 
        week_numbers = torch.arange(0, 20).unsqueeze(0).repeat(batch_size,1)
        inputs = torch.stack([y, log_kilos, week_numbers], dim=2)
        for kilo_range in kilo_ranges:
            
            kilo_inputs = inputs[:,:kilo_range.item(),:]
            outputs, clamp = model(features, ranch_id, class_id, type_id, variety_id, climate_data, kilo_inputs)
            loss = criterion(outputs, y)
            row_results.append(loss.item())

        results.append(row_results)



Fold 1
Epoch [1/200], Loss: 8774458.3313
Epoch [2/200], Loss: 8129256.7048
Epoch [3/200], Loss: 6894883.7726
Epoch [4/200], Loss: 6364594.4352
Epoch [5/200], Loss: 6262899.5512
Epoch [6/200], Loss: 6193039.6672
Epoch [7/200], Loss: 5888283.8720


KeyboardInterrupt: 